<a href="https://colab.research.google.com/github/ds-kdas/Hackathon---Club-Mahindra-customer-spend-prediction/blob/master/Text_summarization_using_AutoNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Configure git repo with username**

In [ ]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 0s (19.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-pr

In [ ]:
!git config --global user.email "vimal.kumar477@gmail.com"
!git config --global user.name "Vimal0307"

**Import required packages**

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd

**Function to extract and map training and validation data**

In [ ]:
data, info = tfds.load('cnn_dailymail', with_info=True)
dataset_train, dataset_test = data['train'], data['test']
def map_fn(x):
    strings = [x['highlights']]
    x['highlights'] = tf.strings.join(strings, separator=' ')
    strings1 = [x['article']]
    x['article'] = tf.strings.join(strings1, separator=' ')
    return x


**Converting Tensorflow Dataset to CSV for AutoNLP- Training Data**

In [ ]:
articles_train = []
highlights_train = []
dataset_train = dataset_train.map(map_fn)
for i,j in zip(range(len(dataset_train)),tfds.as_numpy(dataset_train)):
  if i ==0:
      articles_train.append(str(j['article'][110:])) # Slice the text till 110th index as it containes unwanted text
  else:
    articles_train.append(str(j['article']))
dataset_train = dataset_train.map(map_fn)
for i,j in zip(range(len(dataset_train)),tfds.as_numpy(dataset_train)):
  if i ==0:
      highlights_train.append(str(j['highlights']))
  else:
    highlights_train.append(str(j['highlights']))
print(len(highlights_train))
#Create a data frame whyich can be converted as a CSV
data = pd.DataFrame(
    {'text': articles_train,
     'target': highlights_train
    })
train_csv = data.to_csv("train_text.csv")

287113


**Converting Tensorflow Dataset to CSV for AutoNLP- Testing/Validation Data**

In [ ]:
articles_test = []
highlights_test = []
dataset_test = dataset_test.map(map_fn)
for i,j in zip(range(len(dataset_test)),tfds.as_numpy(dataset_test)):
  if i ==0:
      articles_test.append(str(j['article'][110:]))
  else:
    articles_test.append(str(j['article']))
dataset_test = dataset_test.map(map_fn)
for i,j in zip(range(len(dataset_test)),tfds.as_numpy(dataset_test)):
  if i ==0:
      highlights_test.append(str(j['highlights']))
  else:
    highlights_test.append(str(j['highlights']))
print(len(highlights_test))
data = pd.DataFrame(
    {'text': articles_test,
     'target': highlights_test
    })
test_csv = data.to_csv("test_text.csv")

11490


**Install AutoNLP**

In [ ]:
!pip install -U autonlp

     |████████████████████████████████| 61kB 5.3MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing installation: prettytable 2.1.0
    Uninstalling prettytable-2.1.0:
      Successfully uninstalled prettytable-2.1.0


**Create account in hugging face and get API key from settings and login AutoNLP**

In [ ]:
!autonlp login --api-key <API KEY>

> INFO    🗝 Successfully logged in as Vimal0703
> INFO    🗝 Storing credentials in:  /root/.autonlp
Welcome to 🤗 AutoNLP! Start by creating a project: autonlp create_project


**Create a AutoNLP Project**

In [ ]:
! autonlp create_project --name summarization_model --language en --task summarization --max_models 5


> INFO    Creating project: summarization_model with task: summarization
> INFO    🗝 Retrieving credentials from config...
> INFO    🤙 Project 'summarization_model' already exists, it was loaded successfully.
> INFO    🔄 Refreshing uploaded files information...
> INFO    🔄 Refreshing models information...
> INFO    🔄 Refreshing cost information...
AutoNLP Project (id # 204)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 • Name:        summarization_model
 • Owner:       Vimal0703
 • Status:      ✅ Successfully queued your models for training!
 • Task:        Summarization
 • Created at:  2021-05-19 12:18 Z
 • Last update: 2021-05-19 13:13 Z

💰 Project current cost: USD 125.00

~~~~~~~~~~~~~~ Files ~~~~~~~~~~~~~~

Dataset ID:
Vimal0703/autonlp-data-summarization_model

📁 train_text.csv (id # 253)
   • Split:             train
   • Processing status: ✅ Success!
   • Last update:       2021-05-19 12:48 Z
📁 test_text.csv (id # 254)
   • Split:             valid
   • Processing status: ✅ Success!
   

**Preparing Dataset for AutoNLP**

Training Data

In [ ]:
!autonlp upload --project summarization_model --split train \
            --col_mapping text:text,target:target \
            --files '/content/train_text.csv'

> INFO    Uploading files for project: summarization_model
> INFO    🗝 Retrieving credentials from config...
> INFO    ☁ Retrieving project 'summarization_model' from AutoNLP...
> INFO    🔄 Refreshing uploaded files information...
> INFO    🔄 Refreshing models information...
> INFO    🔄 Refreshing cost information...
> INFO    ✅ Successfully loaded project: 'summarization_model'!
> INFO    Mapping: {'text': 'text', 'target': 'target'}
> INFO    [1/1] 🔎 Validating /content/train_text.csv and column mapping...
> INFO    [1/1] 📦 Copying /content/train_text.csv to /root/.huggingface/autonlp/projects/Vimal0703/autonlp-data-summarization_model/raw/train_text.csv...
> INFO    ☁ Uploading files to the dataset hub...
> INFO    ✅ Successfully uploaded  the files!
> INFO    [1/1] 📁 Registering file train_text.csv into project 'train_text.csv'...
> INFO    [1/1] ✅ Success!
🎉 Yupee! Your files have been uploaded.
Once you're done, starting a training here: autonlp train --project summarization_mode

Validation Data

In [ ]:
!autonlp upload --project summarization_model --split valid \
            --col_mapping text:text,target:target \
            --files '/content/test_text.csv'

> INFO    Uploading files for project: summarization_model
> INFO    🗝 Retrieving credentials from config...
> INFO    ☁ Retrieving project 'summarization_model' from AutoNLP...
> INFO    🔄 Refreshing uploaded files information...
> INFO    🔄 Refreshing models information...
> INFO    🔄 Refreshing cost information...
> INFO    ✅ Successfully loaded project: 'summarization_model'!
> INFO    Mapping: {'text': 'text', 'target': 'target'}
> INFO    [1/1] 🔎 Validating /content/test_text.csv and column mapping...
> INFO    [1/1] 📦 Copying /content/test_text.csv to /root/.huggingface/autonlp/projects/Vimal0703/autonlp-data-summarization_model/raw/test_text.csv...
> INFO    ☁ Uploading files to the dataset hub...
> INFO    ✅ Successfully uploaded  the files!
> INFO    [1/1] 📁 Registering file test_text.csv into project 'test_text.csv'...
> INFO    [1/1] ✅ Success!
🎉 Yupee! Your files have been uploaded.
Once you're done, starting a training here: autonlp train --project summarization_model


**Train AutoNLP**

In [ ]:
!autonlp train --project summarization_model

> INFO    Starting Training For Project: summarization_model
> INFO    🗝 Retrieving credentials from config...
> INFO    ☁ Retrieving project 'summarization_model' from AutoNLP...
> INFO    🔄 Refreshing uploaded files information...
> INFO    🔄 Refreshing models information...
> INFO    🔄 Refreshing cost information...
> INFO    ✅ Successfully loaded project: 'summarization_model'!
> INFO    🔄 Refreshing uploaded files information...
> INFO    🔄 Refreshing models information...
> INFO    🔄 Refreshing cost information...
> INFO    🔎 Calculating a cost estimate for the training...
Traceback (most recent call last):
  File "/usr/lib/python3.7/subprocess.py", line 964, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
  File "/usr/lib/python3.7/subprocess.py", line 1715, in _communicate
    ready = selector.select(timeout)
  File "/usr/lib/python3.7/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt

During han

**Monitor Training progress**

In [ ]:
! autonlp project_info --name summarization_model

> INFO    Fetching info for project: summarization_model
> INFO    🗝 Retrieving credentials from config...
> INFO    ☁ Retrieving project 'summarization_model' from AutoNLP...
> INFO    🔄 Refreshing uploaded files information...
> INFO    🔄 Refreshing models information...
> INFO    🔄 Refreshing cost information...
> INFO    ✅ Successfully loaded project: 'summarization_model'!
AutoNLP Project (id # 204)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 • Name:        summarization_model
 • Owner:       Vimal0703
 • Status:      ✅ Successfully queued your models for training!
 • Task:        Summarization
 • Created at:  2021-05-19 12:18 Z
 • Last update: 2021-05-19 13:13 Z

💰 Project current cost: USD 125.00

~~~~~~~~~~~~~~ Files ~~~~~~~~~~~~~~

Dataset ID:
Vimal0703/autonlp-data-summarization_model

📁 train_text.csv (id # 253)
   • Split:             train
   • Processing status: ✅ Success!
   • Last update:       2021-05-19 12:48 Z
📁 test_text.csv (id # 254)
   • Split:             valid
   • Pr

In [ ]:
!autonlp predict --project summarization_model --model_id 204667 --sentence "At the beginning of the 21st century China had no high-speed railways.Slow and often uncomfortable trains plodded across this vast country, with low average speeds making journeys such as Shanghai-Beijing a test of endurance.Today, it's a completely different picture. The world's most populous nation has -- by some distance -- the world's largest network of high-speed railways."

> INFO    🗝 Retrieving credentials from config...
{'error': 'Model Vimal0703/autonlp-summarization_model-204667 is currently loading', 'estimated_time': 35.669496}
